In [1]:
import os, sys, multiprocessing as mp, pandas as pd, requests, re
from bs4 import BeautifulSoup
pd.set_option('max_colwidth', 200)

In [2]:
r = requests.get('https://deckstats.net/decks/99839/957744-muldrotha-edh?lng=en')
r.content
soup = BeautifulSoup(r.content, 'lxml')

html_data = soup.find_all('div', {'id': 'deck_overview_cards'})

raw_data = str(html_data[0])
indices = [m.start() for m in re.finditer('target="_blank">', raw_data)]
card_list = []
for index in indices:
    card_text = raw_data[index+16:index+56]
    card_text = card_text[0:card_text.find(' </a>')].strip()
    card_list.append(card_text)
len(card_list) # len should be lower than total (duplicate basic lands)

132

In [3]:
cached_data = 'muldrotha.csv'
def get_card_data(card_name):
    r = requests.get('https://api.scryfall.com/cards/search/?q=' + card_name)
    return(r.json()) # we _should_ always get the card we need since titles are exact
card_data = []
if not os.path.exists(cached_data):
    pool = mp.Pool()
    card_data = pool.map(get_card_data, card_list)
    pool.close()

In [4]:
card_df = None
if os.path.exists(cached_data):
    card_df =pd.read_csv('muldrotha.csv')
else:
    card_df = pd.DataFrame(list(map(lambda x: x['data'][0], card_data)))
    card_df = card_df[['name','type_line','cmc','mana_cost','oracle_text','power','toughness','usd']]
    card_df.to_csv('muldrotha.csv', sep = ',', encoding = 'utf-8', index = False)
card_df.head()

,name,type_line,cmc,mana_cost,oracle_text,power,toughness,usd
0,"Muldrotha, the Gravetide",Legendary Creature — Elemental Avatar,6.0,{3}{U}{B}{G},"During each of your turns, you may play up to one permanent card of each permanent type from your graveyard. (If a card has multiple permanent types, choose one as you play it.)",6,6,6.99
1,Abrupt Decay,Instant,2.0,{B}{G},Abrupt Decay can't be countered by spells or abilities.\nDestroy target nonland permanent with converted mana cost 3 or less.,NaN,NaN,4.61
2,Alchemist's Refuge,Land,0.0,,"{T}: Add {C} to your mana pool.\n{G}{U}, {T}: You may cast spells this turn as though they had flash.",NaN,NaN,2.01
3,Arcane Lighthouse,Land,0.0,,"{T}: Add {C} to your mana pool.\n{1}, {T}: Until end of turn, creatures your opponents control lose hexproof and shroud and can't have hexproof or shroud.",NaN,NaN,2.79
4,Archfiend of Depravity,Creature — Demon,5.0,{3}{B}{B},"Flying\nAt the beginning of each opponent's end step, that player chooses up to two creatures he or she controls, then sacrifices the rest.",5,4,0.49


In [5]:
lands = card_df[card_df['type_line'].str.contains('Land', na = False)]
print('Land count: ' + str(len(lands)))
lands[['name','type_line','oracle_text','usd']].sort_values('usd', ascending=False)

Land count: 30


,name,type_line,oracle_text,usd
80,Overgrown Tomb,Land — Swamp Forest,"({T}: Add {B} or {G} to your mana pool.)\nAs Overgrown Tomb enters the battlefield, you may pay 2 life. If you don't, Overgrown Tomb enters the battlefield tapped.",9.66
121,Twilight Mire,Land,"{T}: Add {C} to your mana pool.\n{B/G}, {T}: Add {B}{B}, {B}{G}, or {G}{G} to your mana pool.",9.00
130,Woodland Cemetery,Land,Woodland Cemetery enters the battlefield tapped unless you control a Swamp or a Forest.\n{T}: Add {B} or {G} to your mana pool.,6.87
39,Flooded Grove,Land,"{T}: Add {C} to your mana pool.\n{G/U}, {T}: Add {G}{G}, {G}{U}, or {U}{U} to your mana pool.",6.11
56,Hinterland Harbor,Land,Hinterland Harbor enters the battlefield tapped unless you control a Forest or an Island.\n{T}: Add {G} or {U} to your mana pool.,5.99
35,Drowned Catacomb,Land,Drowned Catacomb enters the battlefield tapped unless you control an Island or a Swamp.\n{T}: Add {U} or {B} to your mana pool.,4.06
38,Field of Ruin,Land,"{T}: Add {C} to your mana pool.\n{2}, {T}, Sacrifice Field of Ruin: Destroy target nonbasic land an opponent controls. Each player searches his or her library for a basic land card, puts it onto t...",3.65
111,Sunken Ruins,Land,"{T}: Add {C} to your mana pool.\n{U/B}, {T}: Add {U}{U}, {U}{B}, or {B}{B} to your mana pool.",21.38
3,Arcane Lighthouse,Land,"{T}: Add {C} to your mana pool.\n{1}, {T}: Until end of turn, creatures your opponents control lose hexproof and shroud and can't have hexproof or shroud.",2.79
5,Ash Barrens,Land,"{T}: Add {C} to your mana pool.\nBasic landcycling {1} ({1}, Discard this card: Search your library for a basic land card, reveal it, put it into your hand, then shuffle your library.)",2.06


In [6]:
mana_rocks = card_df[card_df['oracle_text'].str.contains('mana pool|Mana pool|{C}{C}', na = False)]
mana_rocks = mana_rocks[-mana_rocks['type_line'].str.contains('Land|land', na = False)]
print('Mana ramp count: ' + str(len(mana_rocks)))
mana_rocks.sort_values('usd', ascending=False)

Mana ramp count: 17


,name,type_line,cmc,mana_cost,oracle_text,power,toughness,usd
45,Gilded Lotus,Artifact,5.0,{5},{T}: Add three mana of any one color to your mana pool.,NaN,NaN,8.99
8,Birds of Paradise,Creature — Bird,1.0,{G},Flying\n{T}: Add one mana of any color to your mana pool.,0,1,7.07
63,"Kruphix, God of Horizons",Legendary Enchantment Creature — God,5.0,{3}{G}{U},"Indestructible\nAs long as your devotion to green and blue is less than seven, Kruphix isn't a creature.\nYou have no maximum hand size.\nIf unused mana would empty from your mana pool, that mana ...",4,7,6.57
29,Deathrite Shaman,Creature — Elf Shaman,1.0,{B/G},"{T}: Exile target land card from a graveyard. Add one mana of any color to your mana pool.\n{B}, {T}: Exile target instant or sorcery card from a graveyard. Each opponent loses 2 life.\n{G}, {T}: ...",1,2,4.40
24,Cryptolith Rite,Enchantment,2.0,{1}{G},"Creatures you control have ""{T}: Add one mana of any color to your mana pool.""",NaN,NaN,2.99
117,Thran Dynamo,Artifact,4.0,{4},{T}: Add {C}{C}{C} to your mana pool.,NaN,NaN,2.94
34,Dimir Signet,Artifact,2.0,{2},"{1}, {T}: Add {U}{B} to your mana pool.",NaN,NaN,1.62
18,Commander's Sphere,Artifact,3.0,{3},{T}: Add to your mana pool one mana of any color in your commander's color identity.\nSacrifice Commander's Sphere: Draw a card.,NaN,NaN,1.29
54,Hedron Archive,Artifact,4.0,{4},"{T}: Add {C}{C} to your mana pool.\n{2}, {T}, Sacrifice Hedron Archive: Draw two cards.",NaN,NaN,0.58
71,Mind Stone,Artifact,2.0,{2},"{T}: Add {C} to your mana pool.\n{1}, {T}, Sacrifice Mind Stone: Draw a card.",NaN,NaN,0.46


In [9]:
card_draw = card_df[card_df['oracle_text'].str.contains('Draw|draw', na = False)]
print('Card draw count: ' + str(len(card_draw)))
card_draw[['name','type_line','mana_cost','cmc','oracle_text','usd']].sort_values('cmc', ascending=False)

Card draw count: 26


,name,type_line,mana_cost,cmc,oracle_text,usd
19,Consecrated Sphinx,Creature — Sphinx,{4}{U}{U},6.0,"Flying\nWhenever an opponent draws a card, you may draw two cards.",13.75
72,Mulldrifter,Creature — Elemental,{4}{U},5.0,"Flying\nWhen Mulldrifter enters the battlefield, draw two cards.\nEvoke {2}{U} (You may cast this spell for its evoke cost. If you do, it's sacrificed when it enters the battlefield.)",1.77
54,Hedron Archive,Artifact,{4},4.0,"{T}: Add {C}{C} to your mana pool.\n{2}, {T}, Sacrifice Hedron Archive: Draw two cards.",0.58
105,Solemn Simulacrum,Artifact Creature — Golem,{4},4.0,"When Solemn Simulacrum enters the battlefield, you may search your library for a basic land card, put that card onto the battlefield tapped, then shuffle your library.\nWhen Solemn Simulacrum dies...",3.67
61,"Kiora, the Crashing Wave",Legendary Planeswalker — Kiora,{2}{G}{U},4.0,"+1: Until your next turn, prevent all damage that would be dealt to and dealt by target permanent an opponent controls.\n−1: Draw a card. You may play an additional land this turn.\n−5: You get an...",2.30
116,Thirst for Knowledge,Instant,{2}{U},3.0,Draw three cards. Then discard two cards unless you discard an artifact card.,0.69
95,Secrets of the Dead,Enchantment,{2}{U},3.0,"Whenever you cast a spell from your graveyard, draw a card.",0.11
18,Commander's Sphere,Artifact,{3},3.0,{T}: Add to your mana pool one mana of any color in your commander's color identity.\nSacrifice Commander's Sphere: Draw a card.,1.29
32,Dimir Cluestone,Artifact,{3},3.0,"{T}: Add {U} or {B} to your mana pool.\n{U}{B}, {T}, Sacrifice Dimir Cluestone: Draw a card.",0.08
41,Frantic Search,Instant,{2}{U},3.0,"Draw two cards, then discard two cards. Untap up to three lands.",NaN


In [31]:
removal = card_df[card_df['oracle_text'].str.lower().str.contains('destroy target', na = False)]
removal = removal[-removal['type_line'].str.contains('Land')]
print('Targeted removal: ' + str(len(removal)))
removal # this one will require more thought

Targeted removal: 15


,name,type_line,cmc,mana_cost,oracle_text,power,toughness,usd
1,Abrupt Decay,Instant,2.0,{B}{G},Abrupt Decay can't be countered by spells or abilities.\nDestroy target nonland permanent with converted mana cost 3 or less.,NaN,NaN,4.69
2,Acidic Slime,Creature — Ooze,5.0,{3}{G}{G},"Deathtouch (Any amount of damage this deals to a creature is enough to destroy it.)\nWhen Acidic Slime enters the battlefield, destroy target artifact, enchantment, or land.",2,2,0.25
10,Banewhip Punisher,Creature — Human Warrior,3.0,{2}{B},"When Banewhip Punisher enters the battlefield, you may put a -1/-1 counter on target creature.\n{B}, Sacrifice Banewhip Punisher: Destroy target creature that has a -1/-1 counter on it.",2,2,0.09
14,Bone Splinters,Sorcery,1.0,{B},"As an additional cost to cast Bone Splinters, sacrifice a creature.\nDestroy target creature.",NaN,NaN,0.07
22,Caustic Caterpillar,Creature — Insect,1.0,{G},"{1}{G}, Sacrifice Caustic Caterpillar: Destroy target artifact or enchantment.",1,1,0.10
48,Dimir Charm,Instant,2.0,{U}{B},Choose one —\n• Counter target sorcery spell.\n• Destroy target creature with power 2 or less.\n• Look at the top three cards of target player's library. Put one back and the rest into that player...,NaN,NaN,0.09
67,Golgari Charm,Instant,2.0,{B}{G},Choose one —\n• All creatures get -1/-1 until end of turn.\n• Destroy target enchantment.\n• Regenerate each creature you control.,NaN,NaN,0.25
70,"Grimgrin, Corpse-Born",Legendary Creature — Zombie Warrior,5.0,{3}{U}{B},"Grimgrin, Corpse-Born enters the battlefield tapped and doesn't untap during your untap step.\nSacrifice another creature: Untap Grimgrin and put a +1/+1 counter on it.\nWhenever Grimgrin attacks,...",5,5,4.48
74,Hero's Downfall,Instant,3.0,{1}{B}{B},Destroy target creature or planeswalker.,NaN,NaN,2.49
85,Krosan Grip,Instant,3.0,{2}{G},"Split second (As long as this spell is on the stack, players can't cast spells or activate abilities that aren't mana abilities.)\nDestroy target artifact or enchantment.",NaN,NaN,1.49
